# DeepLearning for Audio With Python 2

## 5. Computation in NN
### Why is a neural network needed?
- A single neuron works for linear problems
- Real-world problems are complex
- ANNs can reporduce highly non-linear functions

### The components of an ANN
- Neurons
- Input, Hidden, Output layers
- Weighted connections
- Activation function

### The multilayer perceptron (MLP)
### Computation in MLP
- Weigths
- Net inputs (sum of weughted inputs)
- Activation (output of neurons to next layer)

### Takeaway points
- ANNs works for complex problems
- Computation is distributed
- Signal moves from left to right
- Weights, net inputs and activations

## 6. Implementing a NN from scratch

In [2]:
import numpy as np

class MLP:
    def __init__(self, num_inputs=3, num_hidden=[3, 5], num_outputs=2):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        # init random weights
        self.weigths = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            self.weigths.append(w)
            
    def forward_propagation(self, inputs):
        activations = inputs
        for w in self.weigths:
            # Calculate net inputs
            net_inputs = np.dot(activations, w)
            # Calculate the activations
            activations = self._sigmoid(net_inputs)
        return activations
    
    def _sigmoid(self, x):
        return 1 / (1+np.exp(-x))
    
if __name__=="__main__":
    # create an MLP
    mlp = MLP()
    # create some inputs
    inputs = np.random.randn(mlp.num_inputs)
    # create forward prop
    outputs = mlp.forward_propagation(inputs)
    # print the results
    print(f"The network input is: {inputs}")
    print(f"The network output is: {outputs}")

The network input is: [-0.9804319   1.49632182  1.57710071]
The network output is: [0.82197151 0.90965432]


## 7. Training a NN: Backpropagation and Gradient Descent
### Training a neural network
- Tweak weights of the connections
- Feed training data (input + target) to the network
- Iterative adjustments

1. Get prediction
2. Calculate error
3. Calculate gradient of error function over the weihts
4. Update parameters

### Gradient descent
- Take a step in opposite direction to gradient
- Step = Learning rate

## 8. Training a NN: Implementing back propagation from scratch

In [4]:
import numpy as np
from random import random


class MLP(object):
    def __init__(self, num_inputs=3, hidden_layers=[3, 3], num_outputs=2):
        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs
        # create a generic representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]
        # create random connection weights for the layers
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights
        # save derivatives per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i + 1]))
            derivatives.append(d)
        self.derivatives = derivatives
        # save activations per layer
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations

    def forward_propagate(self, inputs):
        # the input layer activation is just the input itself
        activations = inputs
        # save the activations for backpropogation
        self.activations[0] = activations
        # iterate through the network layers
        for i, w in enumerate(self.weights):
            # calculate matrix multiplication between previous activation and weight matrix
            net_inputs = np.dot(activations, w)
            # apply sigmoid activation function
            activations = self._sigmoid(net_inputs)
            # save the activations for backpropogation
            self.activations[i + 1] = activations
        # return output layer activation
        return activations

    def back_propagate(self, error):
        # iterate backwards through the network layers
        for i in reversed(range(len(self.derivatives))):
            # get activation for previous layer
            activations = self.activations[i+1]
            # apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)
            # reshape delta as to have it as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T
            # get activations for current layer
            current_activations = self.activations[i]
            # reshape activations as to have them as a 2d column matrix
            current_activations = current_activations.reshape(current_activations.shape[0],-1)
            # save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations, delta_re)
            # backpropogate the next error
            error = np.dot(delta, self.weights[i].T)

    def train(self, inputs, targets, epochs, learning_rate):
        # now enter the training loop
        for i in range(epochs):
            sum_errors = 0
            # iterate through all the training data
            for j, input in enumerate(inputs):
                target = targets[j]
                # activate the network!
                output = self.forward_propagate(input)
                error = target - output
                self.back_propagate(error)
                # now perform gradient descent on the derivatives
                # (this will update the weights
                self.gradient_descent(learning_rate)
                # keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)
            # Epoch complete, report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(items), i+1))
        print("Training complete!")
        print("=====")

    def gradient_descent(self, learningRate=1):
        # update the weights by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives = self.derivatives[i]
            weights += derivatives * learningRate

    def _sigmoid(self, x):
        y = 1.0 / (1 + np.exp(-x))
        return y

    def _sigmoid_derivative(self, x):
        return x * (1.0 - x)

    def _mse(self, target, output):
        return np.average((target - output) ** 2)

if __name__ == "__main__":
    # create a dataset to train a network for the sum operation
    items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])
    targets = np.array([[i[0] + i[1]] for i in items])
    # create a Multilayer Perceptron with one hidden layer
    mlp = MLP(2, [5], 1)
    # train network
    mlp.train(items, targets, 50, 0.1)
    # create dummy data
    input = np.array([0.3, 0.1])
    target = np.array([0.4])
    # get a prediction
    output = mlp.forward_propagate(input)
    print()
    print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))

Error: 0.05110146084713876 at epoch 1
Error: 0.042284148918332945 at epoch 2
Error: 0.04172700946709875 at epoch 3
Error: 0.04116679017115802 at epoch 4
Error: 0.04055467136758766 at epoch 5
Error: 0.039841674801823435 at epoch 6
Error: 0.03897622494198928 at epoch 7
Error: 0.037903320411982085 at epoch 8
Error: 0.03656605569445363 at epoch 9
Error: 0.03491063686141418 at epoch 10
Error: 0.03289601977863392 at epoch 11
Error: 0.030508171621620877 at epoch 12
Error: 0.02777601766283063 at epoch 13
Error: 0.024781957090269875 at epoch 14
Error: 0.021657801075640047 at epoch 15
Error: 0.018561849893586235 at epoch 16
Error: 0.01564422617482242 at epoch 17
Error: 0.013016281324575357 at epoch 18
Error: 0.010736863741329827 at epoch 19
Error: 0.00881664999224095 at epoch 20
Error: 0.007232689236455397 at epoch 21
Error: 0.005944264322785686 at epoch 22
Error: 0.004905036569418351 at epoch 23
Error: 0.004070393234275126 at epoch 24
Error: 0.0034009577011306844 at epoch 25
Error: 0.0028636734